In [28]:
import sys
import tensorflow_data_validation as tfdv
import pandas as pd
from ast import literal_eval

#seems like there's some dependency conflicts
!{sys.executable} -m pip install tensorflow-data-validation
!{sys.executable} -m pip install ipython

  Using cached joblib-0.14.1-py2.py3-none-any.whl (294 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-learn 1.1.0 requires joblib>=1.0.0, but you have joblib 0.14.1 which is incompatible.
You should consider upgrading via the 'd:\remla-baseline-project\venv\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'd:\remla-baseline-project\venv\scripts\python.exe -m pip install --upgrade pip' command.


## Loading Data

In [20]:
def read_data(filename):
    """
    Read, store and return the data from the given filename.

    :param filename: filename of where the data is saved.
    :return: the data in the form of a dataframe.
    """
    data = pd.read_csv(filename, sep='\t', dtype={'title': str, 'tags': object})
    data = data[['title', 'tags']]
    data['tags'] = data['tags'].apply(literal_eval)
    return data

#For some reason it requires this astype for pyarrow
train = read_data('../data/train.tsv')
train['title'] = train['title'].astype(str)
train['tags'] = train['tags'].astype(str)

validation = read_data('../data/validation.tsv')
validation['title'] = validation['title'].astype(str)
validation['tags'] = validation['tags'].astype(str)

test = pd.read_csv('../data/test.tsv', sep='\t', dtype={'title': str})
test = test[['title']]
test['title'] = test['title'].astype(str)

## Data Visualization

In [21]:
train_stats = tfdv.generate_statistics_from_dataframe(train)
tfdv.visualize_statistics(train_stats)

validation_stats = tfdv.generate_statistics_from_dataframe(validation)
tfdv.visualize_statistics(validation_stats)

test_stats = tfdv.generate_statistics_from_dataframe(test)
tfdv.visualize_statistics(test_stats)

## Comparing Data and Validation

In [22]:
tfdv.visualize_statistics(rhs_statistics=train_stats, lhs_statistics=validation_stats, rhs_name="TRAIN", lhs_name="TEST")

# Data Validation

In [23]:
train_schema = tfdv.infer_schema(train_stats)
validation_schema = tfdv.infer_schema(validation_stats)
test_schema = tfdv.infer_schema(test_stats)

In [24]:
tfdv.display_schema(train_schema)
tfdv.display_schema(validation_schema)
tfdv.display_schema(test_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'title',BYTES,required,,-
'tags',BYTES,required,,-


,Type,Presence,Valency,Domain
Feature name,,,,
'title',BYTES,required,,-
'tags',BYTES,required,,-


,Type,Presence,Valency,Domain
Feature name,,,,
'title',BYTES,required,,-


In [25]:
train_anomalies = tfdv.validate_statistics(train_stats, schema=train_schema)
validation_anomalies = tfdv.validate_statistics(validation_stats, schema=validation_schema)
test_anomalies = tfdv.validate_statistics(test_stats, schema=test_schema)

In [26]:
tfdv.display_anomalies(train_anomalies)
tfdv.display_anomalies(validation_anomalies)
tfdv.display_anomalies(test_anomalies)